In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [3]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Focus on important and likely scenarios, important scenario is the scenario that users are more likely to encounter it. 
- Minimize the appearance of rare scenarios. 
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and corresponding use case.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""
FILTER_SYSTEM_PROMPT="""
You will be provide with a use case and a list of test scenario.
Based on information in the use case flow, define what test scenario is necessary to test the use case and remove duplicate test scenarios.
Remove test case test system load error that not be mentioned in use case such as: Load Failure, System Error,Network Error,Non-Existent ...(and use case not mentioned these flow)
Make sure test scenarios filted cover all the flow of use case and every scenarios in the response is unique.
Return scenario name only.
"""


In [4]:
usecase_name = "Search for Books, Seminars, and Library Staff Members"
project_name = "Library"
main_flow_prompt="""
functionName:
    Search for Books, Seminars, and Library Staff Members
objective:
    Allow library users to search for books, seminars, and library staff members
Descriptive sentences related to the function:
    - The Library Management System will be PC-based with internet, allowing library users to search for books, seminars, and library staff members
    - Library user is allowed to use the DLSSYSTEM only for searching book records
"""
prompt_all = """
functionName:
    Search for Books, Seminars, and Library Staff Members
objective:
    Allow library users to search for books, seminars, and library staff members
Descriptive sentences related to the function:
    - The Library Management System will be PC-based with internet, allowing library users to search for books, seminars, and library staff members
    - Library user is allowed to use the DLSSYSTEM only for searching book records
"""

In [5]:
for i in range(1,2):
    print(f"{project_name}\\{usecase_name}-{i}.txt")

Library\Search for Books, Seminars, and Library Staff Members-1.txt


In [6]:

for i in range(1,2):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  filtercontent = ""
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 1
{'Search field': {'condition': {'valid': 'must be used for searching book records', 'invalid': 'used for purposes other than searching book records'}, 'type': 'text field'}}
Scenario Name: Search for Non-Book Records

functionName:
    Search for Books, Seminars, and Library Staff Members
objective:
    Allow library users to search for books, seminars, and library staff members
Descriptive sentences related to the function:
    - The Library Management System will be PC-based with internet, allowing library users to search for books, seminars, and library staff members
    - Library user is allowed to use the DLSSYSTEM only for searching book records


main_gpt_response:1. Search for Books
2. Search for Seminars
3. Search for Library Staff Members

filtercontent: 
Condition Scenario: Scenario Name: Search for Non-Book Records
 Flow cover scenarios: 1. Search for Books
2. Search for Seminars
3. Search for Library Staff Members
filter_gpt_response 
1. Search for Books
2. Search f

In [7]:
usecase_name = "Display Information of E-Book Being Downloaded"
project_name = "Library"
main_flow_prompt="""
    functionName:
      Display Information of E-Book Being Downloaded
    objective:
      Display the information of the e-book being downloaded including ISBN, title, and location
    Descriptive sentences related to the function:
      - When downloading the books, the system shall display the information of the e-book which is just being downloaded including: ISBN, title, location
"""
prompt_all = """
    functionName:
      Display Information of E-Book Being Downloaded
    objective:
      Display the information of the e-book being downloaded including ISBN, title, and location
    Descriptive sentences related to the function:
      - When downloading the books, the system shall display the information of the e-book which is just being downloaded including: ISBN, title, location
"""

In [8]:

for i in range(1,2):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  filtercontent = ""
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 1
{'ISBN': {'condition': {'valid': 'must be displayed during download', 'invalid': 'not displayed'}, 'type': 'text field'}, 'Title': {'condition': {'valid': 'must be displayed during download', 'invalid': 'not displayed'}, 'type': 'text field'}, 'Location': {'condition': {'valid': 'must be displayed during download', 'invalid': 'not displayed'}, 'type': 'text field'}}
1. Scenario: ISBN Not Displayed During E-Book Download
2. Scenario: Title Not Displayed During E-Book Download
3. Scenario: Location Not Displayed During E-Book Download

    functionName:
      Display Information of E-Book Being Downloaded
    objective:
      Display the information of the e-book being downloaded including ISBN, title, and location
    Descriptive sentences related to the function:
      - When downloading the books, the system shall display the information of the e-book which is just being downloaded including: ISBN, title, location


main_gpt_response:1. Successful Display of E-Book Information 

In [9]:
usecase_name = "Temperature change"
project_name = "THEMAS"
main_flow_prompt="""
    If the temperature change is requested, then the determine heating/ cooling mode process is activated and makes a heating/ cooling request.
"""
prompt_all = """
    If the temperature change is requested, then the determine heating/ cooling mode process is activated and makes a heating/ cooling request.
"""

In [10]:

for i in range(1,2):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  filtercontent = ""
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 1
{}

    If the temperature change is requested, then the determine heating/ cooling mode process is activated and makes a heating/ cooling request.


main_gpt_response:1. Scenario: Temperature Increase Request Triggers Heating Mode
2. Scenario: Temperature Decrease Request Triggers Cooling Mode

filtercontent: 
 Flow cover scenarios: 1. Scenario: Temperature Increase Request Triggers Heating Mode
2. Scenario: Temperature Decrease Request Triggers Cooling Mode
filter_gpt_response 
1. Scenario: Temperature Increase Request Triggers Heating Mode
2. Scenario: Temperature Decrease Request Triggers Cooling Mode
An error occurred while writing to the file: [Errno 2] No such file or directory: 'THEMAS\\Temperature change-1.txt'


In [11]:
usecase_name = "Temperature change"
project_name = "THEMAS"
main_flow_prompt="""
    If the current temperature is strictly less than the lower value of the valid temperature range or if if the received temperature value is strictly greater than the upper value of the valid temperture range, then the THEMAS system shall identify the current temperature value as invalid temperature and shall output an invalid temperature status.
"""
prompt_all = """
    If the current temperature is strictly less than the lower value of the valid temperature range or if if the received temperature value is strictly greater than the upper value of the valid temperture range, then the THEMAS system shall identify the current temperature value as invalid temperature and shall output an invalid temperature status.
"""

In [12]:

for i in range(1,2):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  filtercontent = ""
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 1
{'Temperature value': {'condition': {'valid': 'is within the valid temperature range', 'invalid': 'is strictly less than the lower value of the valid temperature range or is strictly greater than the upper value of the valid temperature range'}, 'type': 'text field'}}
Scenario 1: Temperature Value Below Lower Limit Test  
Scenario 2: Temperature Value Above Upper Limit Test

    If the current temperature is strictly less than the lower value of the valid temperature range or if if the received temperature value is strictly greater than the upper value of the valid temperture range, then the THEMAS system shall identify the current temperature value as invalid temperature and shall output an invalid temperature status.


main_gpt_response:1. Temperature Below Valid Range Scenario
2. Temperature Above Valid Range Scenario
3. Temperature Within Valid Range Scenario

filtercontent: 
Condition Scenario: Scenario 1: Temperature Value Below Lower Limit Test  
Scenario 2: Temperature V

In [21]:
usecase_name = "Temperature change"
project_name = "THEMAS"
main_flow_prompt="""
    Temperature values that do not exceed these limits shall be output for subsequent processing.
"""
prompt_all = """
    Temperature values that do not exceed these limits shall be output for subsequent processing.
"""

In [22]:

for i in range(1,2):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  filtercontent = ""
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 1
{}

    Temperature values that do not exceed these limits shall be output for subsequent processing.


main_gpt_response:Given the information and the rules provided, this flow describes a process where temperature values are checked against certain limits, and only those that do not exceed these limits are passed on for further processing. Based on this, here is the scenario:

1. Temperature values within acceptable limits are processed successfully.

filtercontent: 
 Flow cover scenarios: Given the information and the rules provided, this flow describes a process where temperature values are checked against certain limits, and only those that do not exceed these limits are passed on for further processing. Based on this, here is the scenario:

1. Temperature values within acceptable limits are processed successfully.
filter_gpt_response 
1. Temperature values within acceptable limits are processed successfully.
An error occurred while writing to the file: [Errno 2] No such fi

In [19]:
usecase_name = "Temperature change"
project_name = "THEMAS"
main_flow_prompt="""
    If this condition is true, then this module shall output a request to turn on the heating unit in case LO = TLT.
"""
prompt_all = """
    If this condition is true, then this module shall output a request to turn on the heating unit in case LO = TLT.
"""

In [20]:

for i in range(1,2):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  filtercontent = ""
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 1
{}

    If this condition is true, then this module shall output a request to turn on the heating unit in case LO = TLT.


main_gpt_response:Based on the provided information, the main flow of the use case involves a condition check that leads to an action (turning on the heating unit) if a specific condition (LO = TLT) is met. Given the rules for predicting scenarios and focusing on important and likely scenarios while minimizing rare ones, here are the scenarios:

1. Heating Unit Activation Scenario: This scenario covers the condition where LO equals TLT, leading to the activation of the heating unit as described in the flow.

2. No Action Scenario: This scenario occurs when the condition LO = TLT is not met, resulting in no request to turn on the heating unit.

filtercontent: 
 Flow cover scenarios: Based on the provided information, the main flow of the use case involves a condition check that leads to an action (turning on the heating unit) if a specific condition (LO = TLT

In [29]:
usecase_name = "Temperature change"
project_name = "THEMAS"
main_flow_prompt="""
    The heating/cooling unit shall have no real time delay when these statuses are sent to the THEMAS system.
"""
prompt_all = """
    The heating/cooling unit shall have no real time delay when these statuses are sent to the THEMAS system.
"""

In [30]:

for i in range(1,2):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  filtercontent = ""
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 1
{}

    The heating/cooling unit shall have no real time delay when these statuses are sent to the THEMAS system.


main_gpt_response:Based on the provided information, this flow describes a specific behavior of a heating/cooling unit in relation to the THEMAS system, focusing on the real-time update of statuses without delay. Given the constraints and guidance for predicting scenarios, it appears that this flow inherently suggests only one main scenario due to the lack of variability in user actions or conditions that could alter the outcome. Therefore, the scenario predicted is:

1. Real-time Status Update from Heating/Cooling Unit to THEMAS System

filtercontent: 
 Flow cover scenarios: Based on the provided information, this flow describes a specific behavior of a heating/cooling unit in relation to the THEMAS system, focusing on the real-time update of statuses without delay. Given the constraints and guidance for predicting scenarios, it appears that this flow inherently s

In [25]:
usecase_name = "Temperature change"
project_name = "THEMAS"
main_flow_prompt="""
    Each thermostat shall have a unique identifier by which that thermostat is identified in the THEMAS system.
"""
prompt_all = """
    Each thermostat shall have a unique identifier by which that thermostat is identified in the THEMAS system.
"""

In [26]:

for i in range(1,2):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  filtercontent = ""
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 1
{'Unique identifier': {'condition': {'valid': 'must be unique within the THEMAS system', 'invalid': 'is not unique within the THEMAS system'}, 'type': 'text field'}}
Scenario Name: Verify Thermostat Registration with Duplicate Unique Identifier

    Each thermostat shall have a unique identifier by which that thermostat is identified in the THEMAS system.


main_gpt_response:1. Successful Thermostat Registration in THEMAS System

filtercontent: 
Condition Scenario: Scenario Name: Verify Thermostat Registration with Duplicate Unique Identifier
 Flow cover scenarios: 1. Successful Thermostat Registration in THEMAS System
filter_gpt_response 
Verify Thermostat Registration with Duplicate Unique Identifier
An error occurred while writing to the file: [Errno 2] No such file or directory: 'THEMAS\\Temperature change-1.txt'


In [27]:
usecase_name = "Temperature change"
project_name = "THEMAS"
main_flow_prompt="""
    When an event occurs, the THEMAS system shall identify the event type and format an appropriate event message.
"""
prompt_all = """
    When an event occurs, the THEMAS system shall identify the event type and format an appropriate event message.
"""

In [28]:

for i in range(1,2):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  filtercontent = ""
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 1
{}

    When an event occurs, the THEMAS system shall identify the event type and format an appropriate event message.


main_gpt_response:1. Event Successfully Identified and Message Formatted Correctly
2. Event Type Not Recognized
3. Event Message Formatting Error

filtercontent: 
 Flow cover scenarios: 1. Event Successfully Identified and Message Formatted Correctly
2. Event Type Not Recognized
3. Event Message Formatting Error
filter_gpt_response 
1. Event Successfully Identified and Message Formatted Correctly
2. Event Type Not Recognized
3. Event Message Formatting Error
An error occurred while writing to the file: [Errno 2] No such file or directory: 'THEMAS\\Temperature change-1.txt'


In [31]:
usecase_name = "Temperature change"
project_name = "THEMAS"
main_flow_prompt="""
    If the temperature change is requested, then the determine heating/cooling mode process is activated and makes a heating/cooling request.
    If the current temperature value is strictly less than the lower value of the valid temperature range  or if the received temperature value is strictly greater than the upper value of the valid temperature range, then the THEMAS system shall identify the current temperature value as an invalid temperature and shall output an invalid temperature status.
    The THEMAS system shall maintain the ON/OFF status of each heating and cooling unit.
    Temperatures that do not exceed these limits shall be output for subsequent processing.
    If this condition is true, then this module shall output a request to turn on the heating unit in case LO = TLT.
    The heating/cooling unit shall have no real time delay when these statuses are sent to the THEMAS system.
    Each thermostat shall have a unique identifier by which that thermostat is identified in the THEMAS system.
    When an event occurs, the THEMAS system shall identify the event type and format an appropriate event message.
"""
prompt_all = """
    If the temperature change is requested, then the determine heating/cooling mode process is activated and makes a heating/cooling request.
    If the current temperature value is strictly less than the lower value of the valid temperature range  or if the received temperature value is strictly greater than the upper value of the valid temperature range, then the THEMAS system shall identify the current temperature value as an invalid temperature and shall output an invalid temperature status.
    The THEMAS system shall maintain the ON/OFF status of each heating and cooling unit.
    Temperatures that do not exceed these limits shall be output for subsequent processing.
    If this condition is true, then this module shall output a request to turn on the heating unit in case LO = TLT.
    The heating/cooling unit shall have no real time delay when these statuses are sent to the THEMAS system.
    Each thermostat shall have a unique identifier by which that thermostat is identified in the THEMAS system.
    When an event occurs, the THEMAS system shall identify the event type and format an appropriate event message.
"""

In [32]:

for i in range(1,2):
  print("TIME "+str(i))
  filtercontent1 = "Use case: " + prompt_all +"\n Test scenarios: \n"
  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  filtercontent = ""
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    filtercontent = "\nCondition Scenario: " + condition_scenario_response
    print(condition_scenario_response)
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent += "\n Flow cover scenarios: " +  main_gpt_response
  print(main_flow_prompt)
  print("\nmain_gpt_response:"+main_gpt_response+"\n")


  print("filtercontent: "+ filtercontent)
  filtercontent1 += filtercontent
  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent1}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 1
{'Current temperature value': {'condition': {'valid': 'strictly less than the lower value of the valid temperature range or strictly greater than the upper value of the valid temperature range', 'invalid': 'within the valid temperature range'}, 'type': 'text field'}}
Scenario 1: "Submit Temperature Within Valid Range"

    If the temperature change is requested, then the determine heating/cooling mode process is activated and makes a heating/cooling request.
    If the current temperature value is strictly less than the lower value of the valid temperature range  or if the received temperature value is strictly greater than the upper value of the valid temperature range, then the THEMAS system shall identify the current temperature value as an invalid temperature and shall output an invalid temperature status.
    The THEMAS system shall maintain the ON/OFF status of each heating and cooling unit.
    Temperatures that do not exceed these limits shall be output for subsequent pr